In [24]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import os

# Replace with the directory containing your test image
test_image_dir = "q"

# Check if the directory and at least one image exist
if not os.path.isdir(test_image_dir):
    raise FileNotFoundError(f"Test image directory not found: {test_image_dir}")
if len(os.listdir(test_image_dir)) == 0:
    raise FileNotFoundError(f"No images found in directory: {test_image_dir}")

# Define a simple DALI pipeline
class SimpleDALIPipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id):
        super(SimpleDALIPipeline, self).__init__(batch_size, num_threads, device_id)
        self.input = fn.readers.file(file_root=test_image_dir, random_shuffle=False)
        self.decode = fn.decoders.image(self.input, device="mixed", output_type=types.RGB)
        self.resize = fn.resize(self.decode, resize_x=224, resize_y=224)

    def define_graph(self):
        images = self.resize
        return images

# Create and test the pipeline
try:
    pipe = SimpleDALIPipeline(batch_size=1, num_threads=2, device_id=0)
    pipe.build()
    output = pipe.run()

    # Get the output as a NumPy array
    images = output[0].as_cpu().as_array()
    print(f"DALI is working correctly. Output image shape: {images.shape}")
except Exception as e:
    print(f"DALI test failed. Error: {e}")


DALI test failed. Error: Critical error when building pipeline:
Error in CPU operator `nvidia.dali.fn.readers.file`,
which was used in the pipeline definition with the following traceback:

  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/qizixxx/mlbook/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/qizixxx/mlbook/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/qizixxx/mlbook/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/qizixxx/mlbook/.venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever
    self._run_once()
  File "/usr/lib/python3.12/asyncio/base_